In [1]:
import os
import sys
%cd ..
print("Current working directory:", os.getcwd())
project_root = os.path.abspath(os.getcwd())
if project_root not in sys.path:
    sys.path.insert(0, project_root)

c:\Users\nicol\OneDrive - University of Bristol\OT2_group_project
Current working directory: c:\Users\nicol\OneDrive - University of Bristol\OT2_group_project


In [12]:
#from opentrons import protocol_api
from opentrons.simulate import get_protocol_api

import numpy as np

import sys
from optobot.automate import OptimisationLoop
from tests.test_colours import test_get_colours

requirements = {"robotType": "OT-2", "apiLevel": "2.16"}

network_connection = True

# Define an experiment name.
experiment_name = "colour_network_experiment"
data_storage_folder = "tests/test_results_data" 
name = f"{data_storage_folder}/{experiment_name}"

# Define the experimental parameters.
# In this experiment, these are RGB colour pigments and water.
liquid_names = ["water", "blue", "yellow", "red"]

# Define the measured parameters.
# In this experiment, these are the RGB values of the experimental products.
measured_parameter_names = ["measured_red", "measured_green", "measured_blue"]

# Set a target measurement.
# In the real experiment, this a set of defined RGB values. 
#For testing purposes, this is the volumes of the input liquids directly (instead of the measurements)
test_target_measurement = [
    14, 
    20, 
    15]

# Define the search space of the experimental parameters.
# In this experiment, this is the range of volumes for RGB colour pigments.
search_space = [[0.0, 30.0], [0.0, 30.0], [0.0, 30.0]]

# Define the well plate dimensions.
wellplate_size = 96
wellplate_shape = (8, 12)  # As (rows, columns).

# Define the total volume in a well.
total_volume = 90.0

# Define the location of the wellplate in the Opentrons OT-2.
# In this experiment, this is slot 5.
# NOTE: More than one well plate can be used.
# NOTE: For example, slots 5 & 8 -> [5, 8]
wellplate_locs = [5]

# Define the population size for optimisation.
# In this experiment, this is defined as 12 -> 12 wells/columns.
population_size = 5

# Define the number of iterations for optimisation.
# In this experiment, this is defined as 8 -> 8 rows.
num_iterations = 2

# Check that the number of iterations and population size are valid.
if population_size * num_iterations > wellplate_size * len(wellplate_locs):
    print("error: not enough wells for defined population and iteration size")
    sys.exit(1)

# Define an objective function for optimisation.
def objective_function(measurements):
    """
    The objective function to be optimised.

    In this experiment, this calculates the squared Euclidean distance
    between the target RGB value and the measured RGB values.

    Parameters
    ----------
    measurements : np.ndarray
        The measured parameter values of the experimental products.

    Returns
    -------
    errors : np.ndarray
        The errors between the target value and the measured values.
    """

    errors = ((measurements - test_target_measurement) ** 2).sum(axis=1)
    return errors


# Define a measurement function for measuring experimental products.
# NOTE: A measurement function does not have to be defined if measurement input is manual.
def measurement_function(
    liquid_volumes,
    iteration_count,
    population_size,
    num_measured_parameters,
    data_dir,
):
    """
    The measurement function for measuring experimental products.

    In this experiment, this uses the "get_colours" function from the
    "optobot.colorimetric.colours" sub-module. The "get_colours" function
    uses a webcam pointing at the OT-2 deck to take a picture and retrieve
    the RGB values of the experimental products.

    Parameters
    ----------
    liquid_volumes : np.ndarray
        The liquid volumes of the experimental products.

    iteration_count : int
        The current iteration.

    population_size : int
        The population size.

    num_measured_parameters : int
        The number of measured parameters.

    data_dir : string
        The directory for storing the experimental data.

    Returns
    -------
    np.ndarray, float[population_size, num_measured_parameters]
        The measured parameter values of the experimental products.
    """

    return test_get_colours(
        iteration_count, population_size, num_measured_parameters, data_dir
    )

def test_measurement_function(
    liquid_volumes,
    iteration_count,
    population_size,
    num_measured_parameters,
    data_dir,
):
    """
    Function that skips the measurement step for testing purposes to check the optimisation works, 
    by using the input liquid volumes directly as the "measurements".
    
    """
    return liquid_volumes[:, 1:]

#def run(protocol: protocol_api.ProtocolContext):
def run(protocol):

    #loading the tips, reservoir and well plate into the program
    tips = protocol.load_labware("opentrons_96_tiprack_1000ul", 1)
    reservoir = protocol.load_labware("nest_12_reservoir_15ml", 2)
    
    plates = {}
    for idx, loc in enumerate(wellplate_locs):
        plates[f"plate_{idx+1}"] = protocol.load_labware("nest_96_wellplate_100ul_pcr_full_skirt", loc)
    
    left_pipette = protocol.load_instrument("p1000_single_gen2", "right", tip_racks=[tips])

    ot2_labware = {"reservoir": reservoir, "plates": plates, "left_pipette": left_pipette}


    model = OptimisationLoop(
        objective_function,
        liquid_names,
        measured_parameter_names,
        population_size,
        name=name,
        measurement_function=test_measurement_function,
        wellplate_shape=wellplate_shape,
        wellplate_locs=wellplate_locs,
        total_volume=total_volume,
        network_connection = network_connection,
        ot2_labware = ot2_labware
    )
    # call particle_swarm, random_forest, or gaussian process
    model.optimise(search_space, optimiser="PSO", num_iterations=num_iterations)


protocol = get_protocol_api("2.16") 
run(protocol)

for command in protocol.commands():
    print(command)


2025-04-06 20:34:43,671 - opentrons.config.robot_configs - WARNING - C:\Users\nicol\.opentrons\robot_settings.json not found. Loading defaults
2025-04-06 20:34:43,677 - opentrons.calibration_storage.ot2.deck_attitude - WARNING - Deck calibration not found.
2025-04-06 20:34:43,682 - opentrons.config.robot_configs - WARNING - C:\Users\nicol\.opentrons\deck_calibration.json not found. Loading defaults
2025-04-06 20:34:43,685 - opentrons.hardware_control.api.API.2775426872960 - INFO - Updating instrument model cache
2025-04-06 20:34:43,687 - opentrons.hardware_control.api.API.2775426872960 - INFO - Skipping configuration on LEFT
2025-04-06 20:34:43,689 - opentrons.hardware_control.api.API.2775426872960 - INFO - Skipping configuration on RIGHT
2025-04-06 20:34:43,692 - opentrons.hardware_control.api.API.2775426872960 - INFO - Instruments found: {<Mount.LEFT: 1>: None, <Mount.RIGHT: 2>: None}
2025-04-06 20:34:43,728 - opentrons.protocol_api.protocol_context - INFO - Loading PipetteNameType.P

plate_1


2025-04-06 20:34:43,947 - opentrons.legacy_broker - INFO - command.DISPENSE: instrument: P1000 Single-Channel GEN2 on right mount, volume: 8.997250787471112, location: Location(point=Point(x=173.88, y=164.74, z=1.92), labware=A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5, is_meniscus=False), rate: 1.0
2025-04-06 20:34:43,947 - opentrons.legacy_broker - INFO - command.TRANSFER: instrument: P1000 Single-Channel GEN2 on right mount, locations: [A3 of NEST 12 Well Reservoir 15 mL on slot 2, A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5], volume: 20.616320994917714, source: A3 of NEST 12 Well Reservoir 15 mL on slot 2, dest: A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
2025-04-06 20:34:43,956 - opentrons.legacy_broker - INFO - command.ASPIRATE: instrument: P1000 Single-Channel GEN2 on right mount, volume: 20.616320994917714, location: Location(point=Point(x=164.88, y=42.78, z=5.55), labware=A3 of NEST 12 Well Reservoir 15 mL on slot 2, is_meniscus=False), r

plate_1


2025-04-06 20:34:44,421 - opentrons.legacy_broker - INFO - command.TRANSFER: instrument: P1000 Single-Channel GEN2 on right mount, locations: [A3 of NEST 12 Well Reservoir 15 mL on slot 2, B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5], volume: 15.313335293138033, source: A3 of NEST 12 Well Reservoir 15 mL on slot 2, dest: B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
2025-04-06 20:34:44,423 - opentrons.legacy_broker - INFO - command.ASPIRATE: instrument: P1000 Single-Channel GEN2 on right mount, volume: 15.313335293138033, location: Location(point=Point(x=164.88, y=42.78, z=5.55), labware=A3 of NEST 12 Well Reservoir 15 mL on slot 2, is_meniscus=False), rate: 1.0
2025-04-06 20:34:44,431 - opentrons.legacy_broker - INFO - command.DISPENSE: instrument: P1000 Single-Channel GEN2 on right mount, volume: 15.313335293138033, location: Location(point=Point(x=155.88, y=155.74, z=1.92), labware=B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5, is_meniscus=False), 

Picking up tip from A1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1
Transferring 25.695989766652588 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
Aspirating 25.695989766652588 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
Dispensing 25.695989766652588 uL into A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
Transferring 50.26253586203111 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
Aspirating 50.26253586203111 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
Dispensing 50.26253586203111 uL into A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
Transferring 31.889043341889405 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
Aspirating 31.889043341889405 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 

## Organized ouptut of example run (2 iterations, population size of 5)

### ITERATION 1:

**(WATER:) Picking up tip from A1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1**

        Transferring 29.679185884964795 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 29.679185884964795 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 29.679185884964795 uL into A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 73.93736129844247 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 73.93736129844247 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 73.93736129844247 uL into A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 31.979160786707297 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 31.979160786707297 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 31.979160786707297 uL into A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 71.95132957508116 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 71.95132957508116 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 71.95132957508116 uL into A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 44.45826763737515 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 44.45826763737515 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 44.45826763737515 uL into A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec

**Dropping tip into Trash Bin on slot 12**

**(BLUE:) Picking up tip from B1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1**

        Transferring 26.846785345496773 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 26.846785345496773 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 26.846785345496773 uL into A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 5.442923196557437 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 5.442923196557437 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 5.442923196557437 uL into A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 23.148732162476843 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 23.148732162476843 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 23.148732162476843 uL into A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 3.6645505995846817 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 3.6645505995846817 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 3.6645505995846817 uL into A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 27.968461585561386 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 27.968461585561386 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 27.968461585561386 uL into A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec

**Dropping tip into Trash Bin on slot 12**

**(YELLOW:) Picking up tip from C1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1**

        Transferring 7.356580544783922 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 7.356580544783922 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 7.356580544783922 uL into A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 3.768142931976759 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 3.768142931976759 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 3.768142931976759 uL into A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 22.035693203421705 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 22.035693203421705 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 22.035693203421705 uL into A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 6.07453819597894 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 6.07453819597894 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 6.07453819597894 uL into A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 14.177595457910833 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 14.177595457910833 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 14.177595457910833 uL into A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec

**Dropping tip into Trash Bin on slot 12**

**(RED + mixing as it's the last:) Picking up tip from D1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1**

        Transferring 26.117448224754515 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 26.117448224754515 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 26.117448224754515 uL into A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                Mixing 3 times with a volume of 20.0 ul
                        Aspirating 20.0 uL from A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 6.851572573023333 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 6.851572573023333 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 6.851572573023333 uL into A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                Mixing 3 times with a volume of 20.0 ul
                        Aspirating 20.0 uL from A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 12.836413847394152 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 12.836413847394152 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 12.836413847394152 uL into A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                Mixing 3 times with a volume of 20.0 ul
                        Aspirating 20.0 uL from A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 8.309581629355216 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 8.309581629355216 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 8.309581629355216 uL into A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                Mixing 3 times with a volume of 20.0 ul
                        Aspirating 20.0 uL from A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 3.395675319152631 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 3.395675319152631 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 3.395675319152631 uL into A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                Mixing 3 times with a volume of 20.0 ul
                        Aspirating 20.0 uL from A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into A5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
**Dropping tip into Trash Bin on slot 12**

### ITERATION 2:

**(WATER:) Picking up tip from E1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1**

        Transferring 29.164651605948038 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 29.164651605948038 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 29.164651605948038 uL into B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 62.46149042302737 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 62.46149042302737 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 62.46149042302737 uL into B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 31.827012826485664 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 31.827012826485664 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 31.827012826485664 uL into B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 63.489000964356315 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 63.489000964356315 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 63.489000964356315 uL into B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 39.65650871505129 from A1 of NEST 12 Well Reservoir 15 mL on slot 2 to B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 39.65650871505129 uL from A1 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 39.65650871505129 uL into B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec

**Dropping tip into Trash Bin on slot 12**

**(BLUE:) Picking up tip from F1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1**

        Transferring 25.78357757544546 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 25.78357757544546 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 25.78357757544546 uL into B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 6.925259149993698 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 6.925259149993698 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 6.925259149993698 uL into B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 23.178801783809078 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 23.178801783809078 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 23.178801783809078 uL into B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 7.926919752225764 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 7.926919752225764 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 7.926919752225764 uL into B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 27.702139929683238 from A2 of NEST 12 Well Reservoir 15 mL on slot 2 to B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 27.702139929683238 uL from A2 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 27.702139929683238 uL into B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec

**Dropping tip into Trash Bin on slot 12**

**(YELLOW:) Picking up tip from G1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1**

        Transferring 12.17541953189992 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 12.17541953189992 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 12.17541953189992 uL into B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 10.745847958954517 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 10.745847958954517 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 10.745847958954517 uL into B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 22.068571094690775 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 22.068571094690775 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 22.068571094690775 uL into B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 9.184860530033264 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 9.184860530033264 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 9.184860530033264 uL into B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 16.524382617850204 from A3 of NEST 12 Well Reservoir 15 mL on slot 2 to B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 16.524382617850204 uL from A3 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 16.524382617850204 uL into B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec

**Dropping tip into Trash Bin on slot 12**

**(RED + mixing as it's the last:) Picking up tip from H1 of Opentrons OT-2 96 Tip Rack 1000 µL on slot 1**

        Transferring 22.87635128670658 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 22.87635128670658 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 22.87635128670658 uL into B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                Mixing 3 times with a volume of 20.0 ul
                        Aspirating 20.0 uL from B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B1 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 9.86740246802442 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 9.86740246802442 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 9.86740246802442 uL into B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                Mixing 3 times with a volume of 20.0 ul
                        Aspirating 20.0 uL from B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B2 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 12.925614295014483 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 12.925614295014483 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 12.925614295014483 uL into B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                Mixing 3 times with a volume of 20.0 ul
                        Aspirating 20.0 uL from B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B3 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 9.399218753384659 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 9.399218753384659 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 9.399218753384659 uL into B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                Mixing 3 times with a volume of 20.0 ul
                        Aspirating 20.0 uL from B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B4 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
        Transferring 6.116968737415267 from A4 of NEST 12 Well Reservoir 15 mL on slot 2 to B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5
                Aspirating 6.116968737415267 uL from A4 of NEST 12 Well Reservoir 15 mL on slot 2 at 274.7 uL/sec
                Dispensing 6.116968737415267 uL into B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                Mixing 3 times with a volume of 20.0 ul
                        Aspirating 20.0 uL from B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Aspirating 20.0 uL from B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec
                        Dispensing 20.0 uL into B5 of NEST 96 Well Plate 100 µL PCR Full Skirt on slot 5 at 274.7 uL/sec

**Dropping tip into Trash Bin on slot 12**
